# Information Retrieval 

Information retrieval is defined as a completely automated procedure that answers to a user query by reviewing a group of documents and producing a sorted document list that ought to be relevant to the user's query criteria.

The information retrieval process can be divided into four distinct phases: indexing,
querying, comparison, and feedback.

IR is widely used in various applications such as search engines, question answering systems, and recommendation systems. It has become increasingly important with the growth of digital information, which has made it challenging for users to find relevant information efficiently and effectively.

## Information Retrieval Search Models

- **Boolean model:** In the Boolean model, a query is represented as a Boolean expression of terms, and a document is either relevant or non-relevant to the query. This model is simple and efficient, but it can be too restrictive and may not account for partial matches or term frequency.

- **Vector space model:** In the vector space model, a document is represented as a vector of term weights, and a query is represented as a vector of term weights as well. The similarity between the query vector and each document vector is computed using a similarity measure, such as cosine similarity. This model is flexible and can handle partial matches and term frequency, but it may suffer from the "curse of dimensionality" and may require normalization and tuning.

- **Probabilistic model:** In the probabilistic model, a document is ranked based on the probability of relevance given the query, which is computed using Bayes' theorem. This model is more robust than the Boolean model and can handle partial matches and term frequency, but it may require tuning and may be sensitive to the choice of priors.

- **Language model:** In the language model, a query is modeled as a language model, which is a probability distribution over terms, and a document is modeled as a language model as well. The relevance of a document to a query is measured using the likelihood ratio of the document language model under the query language model and the background language model. This model is effective in handling uncertainty and partial matches, but it may require tuning and may be sensitive to the choice of smoothing methods.

### Boolean Model

In [44]:
class BooleanRetrievalSystem:
    def __init__(self, documents):
        self.documents = documents
        self.vocab = set()
        self.doc_freqs = []
        self.doc_vectors = []
        self.index()

    def index(self):
        # compute document frequencies and build vocabulary
        self.doc_freqs = [set(doc.split()) for doc in self.documents]
        self.vocab = set().union(*self.doc_freqs)

    def search(self, query, k=10):
        # preprocess query
        #query_terms = set(query.split())
        stop_words = set(stopwords.words('english'))
        stemmer = SnowballStemmer('english')
        query_terms = [stemmer.stem(term.lower()) for term in query.split() if term.lower() not in stop_words]
        
        # compute document scores using boolean model
        scores = []
        for i, doc_freq in enumerate(self.doc_freqs):
            # check if any query terms appear in document
            if any(term in doc_freq for term in query_terms):
                scores.append(i)
        
        # rank documents by score and return top k
        return [self.documents[i] for i in scores[:k]]


In [47]:
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "A quick brown dog jumps over the lazy fox.",
    "The lazy brown dog jumps over the quick fox.",
    "The quick brown fox jumps over the lazy dog once more.",
    "The dog is not so lazy anymore.",
    "The brown dog and the black dog are chasing each other.",
    "I saw a blue car and a green car on my way to work.",
    "The sky is blue but the water is clearer.",
    "Blue is my favorite color, but green is a close second.",
    "The sun is shining and the birds are singing."
]

br = BooleanRetrievalSystem(documents)
results = br.search("lazy dog")

print(results)

['A quick brown dog jumps over the lazy fox.', 'The lazy brown dog jumps over the quick fox.', 'The quick brown fox jumps over the lazy dog once more.', 'The dog is not so lazy anymore.', 'The brown dog and the black dog are chasing each other.']


### Vector space model

In [52]:
import math
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from numpy.linalg import norm
import numpy as np

class RetrievalSystem_VectorSpace:
    def __init__(self, documents):
        self.documents = documents
        self.vocab = set()
        self.doc_freqs = []
        self.doc_vectors = []
        self.index()

    def index(self):
        # compute document frequencies and build vocabulary
        self.doc_freqs = [Counter(doc.split()) for doc in self.documents]
        self.vocab = set().union(*self.doc_freqs)
        # compute document vectors
        for doc_freq in self.doc_freqs:
            doc_vector = [doc_freq.get(term, 0) for term in self.vocab]
            self.doc_vectors.append(doc_vector)

    def search(self, query, k=10):
        stop_words = set(stopwords.words('english'))
        stemmer = SnowballStemmer('english')
        query_terms = [stemmer.stem(term.lower()) for term in query.split() if term.lower() not in stop_words]
        scores = []
        for i, doc_vector in enumerate(self.doc_vectors):
            if any(term in self.doc_freqs[i] for term in query_terms):
                query_vector = [Counter(query.split()).get(term, 0) for term in self.vocab]
                cosine_sim = np.dot(doc_vector, query_vector) / (norm(doc_vector) * norm(query_vector)) if norm(doc_vector) and norm(query_vector) else 0
                scores.append((i, cosine_sim))
        ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)
        return [self.documents[i] for i, _ in ranked_docs[:k] if _ > 0]


In [53]:
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "A quick brown dog jumps over the lazy fox.",
    "The lazy brown dog jumps over the quick fox.",
    "The quick brown fox jumps over the lazy dog once more.",
    "The dog is not so lazy anymore.",
    "The brown dog and the black dog are chasing each other.",
    "I saw a blue car and a green car on my way to work.",
    "The sky is blue but the water is clearer.",
    "Blue is my favorite color, but green is a close second.",
    "The sun is shining and the birds are singing."
]

rs = RetrievalSystem_VectorSpace(documents)
results = rs.search(" lazy dog ")

print(results)


['The dog is not so lazy anymore.', 'A quick brown dog jumps over the lazy fox.', 'The lazy brown dog jumps over the quick fox.', 'The quick brown fox jumps over the lazy dog once more.', 'The brown dog and the black dog are chasing each other.']


## Preprocessing

Pre-processing is a crucial step in information retrieval, as it helps to remove noise from the data and extract relevant features. Some pre-processing techniques include tokenization, stop-word removal, stemming, and normalization.

## Document represenation

Documents can be represented in different ways, such as bag-of-words models, TF-IDF models. 

### Similarity (Scoring) Functions

#### TF-IDF

When we’re looking for document relevancy with a certain word, we want the word to be:

- Common Locally: The word appears many times in the document
- Rare Globally: The word doesn’t appear many times altogether in the corpus.

Documents with a word that is common locally but rare globally are documents that are relevant for the given word. With TF-IDF, we can take into account both the Common Locally and Rare Globally factors of documents when calculating which are the most relevant.

TF-IDF works like magic, it can calculate the most relevant documents just the way we want it to, but it has it's drawbacks.

It doesn’t take document length into account: Let’s say that we have a 1,000 word document containing 1 appearance of the word "soccer" and with 10 appearances of the word "soccer". Which document do you think is more relevant to the word "soccer"? It should be the one with 10 words, because there is a greater chance that the document’s topic is about "soccer" compared to the 1,000 words one.

The Term Frequency is not saturated: We know from the previous section that IDF will penalize common words. But what if there are some documents that naturally have so many common words? The Term Frequency’s value will be big. Because the Term Frequency of the TF-IDF function is not saturated, it will boost the irrelevant documents which contain many common words.
Because of those shortcomings, people consider BM25 as the state-of-the-art similarity function.


#### BM25 

Okapi BM25 (Best Matching 25) is a ranking function used by search engines to evaluate the relevance of documents to a given search query. It is a probabilistic model that calculates the score of a document based on the frequency of query terms within the document, as well as their frequency within the entire collection of documents.

BM25 takes into account the inverse document frequency (IDF) of query terms, as well as the document length and term frequency (TF) of each term within a document.

![alt text](bm25-formula.png "bm25 equation")

We can control how much the function penalizes longer documents by changing the b parameter. If we use a large value for b, then the function will penalize the longer document’s score more.

the parameter k1 will determine how saturated the Term Frequency is. The lower the value of k1 is, the more saturated the Term Frequency is.

#### BM25 Implementation

In [29]:
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

docs[0:1]

["\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"]

In [14]:
!pip install rank-bm25

In [54]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.lower().split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)
     

In [55]:
import string

corpus = [doc.translate(str.maketrans('', '', string.punctuation)).replace('\n',"").lower() for doc in corpus]

corpus[100]

'yeah right sorta like the indian subcontient eh'

In [56]:
query = "american immigration"
tokenized_query = query.lower().split(" ")
bm25.get_top_n(tokenized_query, corpus, n=4)

['i am fairly sure that she could obtain citizenship by making anapplication for it it might require immigration to germany buti am almost certain that once applied for citizenship is inevitablein this casemore interesting only for your propaganda purposes i have said severaltimes now that i dont consider iran particularly exemplary as a goodislamic state we might talk about the rights of people in capitalistsecular third world countries to give other examples of the lack ofrights in third world countries broadly say for example centralamerican secular capitalist countries whose govts the us supportsbut who amnesty international has pointed out are human rights vacua',
 '   analog sf magazine did an article on a similar subject quite a fewyears ago  the question was if an alien spacecraft landed inwashington dc what was the proper organization to deal with it thestate department alien ambassadors the defense department alieninvaders the immigration and naturalization service illegal al

## References

- https://www.simplilearn.com/tutorials/machine-learning-tutorial/information-
- https://www.cs.waikato.ac.nz/~ihw/papers/00-SJC-JL-IHW-Applicationml.pdf
- https://www.geeksforgeeks.org/what-is-information-retrieval/
- https://ris.utwente.nl/ws/portalfiles/portal/5588097/IRModelsTutorial-draft.pdf
- https://towardsdatascience.com/understanding-term-based-retrieval-methods-in-information-retrieval-2be5eb3dde9f
- https://www.infoq.com/articles/similarity-scoring-elasticsearch/
- https://towardsdatascience.com/getting-started-with-elasticsearch-in-python-c3598e718380
- https://iamgeekydude.com/2022/11/21/how-to-build-a-search-engine-using-bm25-ranking/
- https://eudl.eu/pdf/10.4108/eetiot.v8i3.2276